<a href="https://colab.research.google.com/github/Gustavo-dev04/Analise-de-dados-de-acoes-com-ia/blob/main/Processamento_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 IMPORTAÇÕES ESSENCIAIS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import requests
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from google.colab import userdata
import os
import warnings

# 📌 CONFIGURAÇÕES
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (18, 9)

# Configurações
SYMBOL = 'NVDA'
DAYS_TO_FETCH = 365 * 1  # 2 anos de dados
TEST_SIZE = 0.2
LEARNING_RATE = 0.05
EPOCHS = 50000
VOLATILITY_WINDOW = 20  # Janela para cálculo de volatilidade

# 📌 CONFIGURAÇÃO SEGURA DA API KEY (Google Colab)
try:
    NEWS_API_KEY = userdata.get('NEWS_API_KEY')
    print("✅ API Key carregada com sucesso dos Secrets do Colab")
except Exception as e:
    print(f"⚠️ AVISO: {str(e)}")
    print("⚠️ Configure sua API Key nos Secrets do Colab (barra lateral > Secrets)")
    NEWS_API_KEY = input("Cole temporariamente sua API Key da NewsAPI: ").strip()
    if not NEWS_API_KEY:
        print("⚠️ Usando modo simulado - funcionalidades limitadas")
        NEWS_API_KEY = None

# Baixa recursos do NLTK
nltk.download('vader_lexicon', quiet=True)

# ======================
# 📌 1. ANÁLISE DE NOTÍCIAS EM TEMPO REAL
# ======================
class NewsSentimentAnalyzer:
    def __init__(self):
        self.analyzer = SentimentIntensityAnalyzer()
        self.session = requests.Session()

    def fetch_real_news(self, query='NVIDIA', language='en', page_size=1000):
        """Busca notícias reais com fallback simulado"""
        if not NEWS_API_KEY:
            print("⚠️ Modo simulado ativado (sem API Key configurada)")
            return self._get_simulated_news()

        try:
            print(f"\n🔍 Buscando notícias reais sobre {query}...")
            url = f"https://newsapi.org/v2/everything?q={query}&language={language}&pageSize={page_size}&apiKey={NEWS_API_KEY}"
            response = self.session.get(url, timeout=15)
            response.raise_for_status()
            data = response.json()

            if data['status'] != 'ok':
                print(f"⚠️ Problema na API: {data.get('message', 'Erro desconhecido')}")
                return self._get_simulated_news()

            return data['articles']
        except Exception as e:
            print(f"⚠️ Erro ao acessar API: {str(e)}")
            return self._get_simulated_news()

    def _get_simulated_news(self):
        """Fornece dados simulados quando a API falha"""
        return [
            {'title': 'NVIDIA anuncia avanços em IA com novos chips', 'content': 'Inovação revolucionária no mercado', 'publishedAt': datetime.now().isoformat()},
            {'title': 'Ações da NVIDIA em alta após resultados positivos', 'content': 'Crescimento acima das expectativas', 'publishedAt': datetime.now().isoformat()},
            {'title': 'NVIDIA enfrenta desafios na cadeia de suprimentos', 'content': 'Problemas logísticos afetam produção', 'publishedAt': datetime.now().isoformat()}
        ]

    def analyze_sentiment(self, articles):
        """Analisa o sentimento das notícias"""
        if not articles:
            return 0.0  # Retorno neutro se não houver notícias

        sentiments = []
        print("\n📰 Análise de Sentimento das Notícias:")

        for article in articles[:1000]:  # Analisa apenas as 5 primeiras
            text = f"{article.get('title', '')} {article.get('content', '')}"
            vs = self.analyzer.polarity_scores(text)
            compound = vs['compound']

            # Classificação visual
            if compound >= 0.05:
                emoji = "📈"
                sentiment = "POSITIVO"
            elif compound <= -0.05:
                emoji = "📉"
                sentiment = "NEGATIVO"
            else:
                emoji = "➖"
                sentiment = "NEUTRO"

            print(f"{emoji} {sentiment} ({compound:.2f}): {article.get('title','')[:60]}...")
            sentiments.append(compound)

        avg_sentiment = np.mean(sentiments) if sentiments else 0.0
        print(f"\n🔍 Sentimento Médio: {avg_sentiment:.2f} (Escala: -1 a 1)")
        return avg_sentiment

# ======================
# 📌 2. MODELO DE PREVISÃO DE PREÇOS
# ======================
class StockPredictor:
    def __init__(self):
        self.learning_rate = LEARNING_RATE
        self.epochs = EPOCHS
        self.weights = None
        self.bias = None
        self.scaler = RobustScaler()
        self.feature_cols = [
            'close', 'volume', 'ma_5', 'ma_20',
            'volatility_20', 'trend', 'sentiment'
        ]
        self.news_analyzer = NewsSentimentAnalyzer()

    def prepare_data(self, df):
        """Prepara os dados para o modelo garantindo consistência"""
        try:
            # Feature Engineering
            df['return'] = df['close'].pct_change()

            # Médias Móveis
            for window in [5, 20]:
                df[f'ma_{window}'] = df['close'].rolling(window).mean()

            # Volatilidade
            df['volatility_20'] = df['return'].rolling(VOLATILITY_WINDOW).std()

            # Tendência
            df['trend'] = (df['close'] > df['ma_20']).astype(int)

            # Análise de Sentimento
            df['sentiment'] = self.news_analyzer.analyze_sentiment(
                self.news_analyzer.fetch_real_news()
            )

            # Remove valores NA
            df = df.dropna()

            # Garante alinhamento correto entre X e y
            X = df[self.feature_cols].iloc[:-1]  # Remove a última linha para X
            y = df['close'].shift(-1).dropna()   # Remove a primeira linha para y

            # Verificação de consistência
            if len(X) != len(y):
                raise ValueError(f"Número de amostras inconsistente: X={len(X)}, y={len(y)}")

            # Normalização
            X_scaled = self.scaler.fit_transform(X)

            # Divisão Treino-Teste Temporal
            split_idx = int(len(X) * (1 - TEST_SIZE))
            X_train, X_test = X_scaled[:split_idx], X_scaled[split_idx:]
            y_train, y_test = y[:split_idx], y[split_idx:]

            # Adiciona coluna de bias
            X_train = np.c_[np.ones(X_train.shape[0]), X_train]
            X_test = np.c_[np.ones(X_test.shape[0]), X_test]

            return X_train, X_test, y_train, y_test, df

        except Exception as e:
            print(f"⚠️ Erro na preparação: {str(e)}")
            # Fallback sem sentimento se necessário
            self.feature_cols = [col for col in self.feature_cols if col != 'sentiment']
            return self.prepare_data(df)

    def train(self, X, y):
        """Treina o modelo com Gradiente Descendente"""
        try:
            n_samples, n_features = X.shape
            self.weights = np.zeros(n_features)
            self.bias = 0
            cost_history = []

            for epoch in range(self.epochs):
                y_pred = np.dot(X, self.weights) + self.bias

                # Gradientes
                dw = (1/n_samples) * np.dot(X.T, (y_pred - y))
                db = (1/n_samples) * np.sum(y_pred - y)

                # Atualização
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db

                # Cálculo de Custo
                cost = np.mean((y_pred - y)**2)
                cost_history.append(cost)

                if epoch % 1000 == 0:
                    print(f"Época {epoch}: MSE = {cost:.4f}")

            return cost_history

        except Exception as e:
            raise Exception(f"Erro no treinamento: {str(e)}")

    def predict(self, X):
        """Faz previsões"""
        return np.dot(X, self.weights) + self.bias

    def evaluate(self, X_test, y_test):
        """Avalia o modelo com garantia de alinhamento"""
        try:
            y_pred = self.predict(X_test)

            # Cria Series com mesmo índice para alinhamento
            y_pred_series = pd.Series(y_pred, index=y_test.index)

            # Métricas
            mae = mean_absolute_error(y_test, y_pred_series)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred_series))
            r2 = r2_score(y_test, y_pred_series)

            # Plot
            plt.figure(figsize=(18, 9))
            plt.plot(y_test.values, label='Real', color='#1f77b4', linewidth=2)
            plt.plot(y_pred_series.values, label='Previsto', color='#ff7f0e', linestyle='--')

            plt.title(f'Desempenho do Modelo (MAE: ${mae:.2f}, R²: {r2:.2%})', fontsize=16)
            plt.xlabel('Dias')
            plt.ylabel('Preço ($)')
            plt.legend()
            plt.grid(alpha=0.3)
            plt.show()

            return {'mae': mae, 'rmse': rmse, 'r2': r2}

        except Exception as e:
            raise Exception(f"Erro na avaliação: {str(e)}")

    def forecast(self, df, days=7):
        """Previsão para os próximos dias com consistência"""
        try:
            forecasts = []
            dates = []
            last_data = df[self.feature_cols].iloc[-1:].copy()

            for i in range(1, days+1):
                # Prepara dados
                data_scaled = self.scaler.transform(last_data)
                data_scaled = np.c_[np.ones(data_scaled.shape[0]), data_scaled]

                # Previsão
                pred = self.predict(data_scaled)[0]
                forecasts.append(pred)
                dates.append(df.index[-1] + timedelta(days=i))

                # Atualiza para próxima previsão
                new_row = last_data.copy()
                new_row['close'] = pred

                # Atualiza indicadores técnicos
                new_row['ma_5'] = np.mean([*df['close'].iloc[-4:], pred])
                new_row['ma_20'] = np.mean([*df['close'].iloc[-19:], pred])
                new_row['volatility_20'] = df['volatility_20'].iloc[-1]  # Mantém última volatilidade conhecida
                new_row['trend'] = 1 if pred > new_row['ma_20'].values[0] else 0

                # Mantém o mesmo sentimento
                if 'sentiment' in new_row.columns:
                    new_row['sentiment'] = last_data['sentiment'].values[0]

                last_data = new_row

            # Cria DataFrame com previsões
            forecast_df = pd.DataFrame({
                'date': dates,
                'forecast': forecasts
            }).set_index('date')

            # Plot
            plt.figure(figsize=(18, 9))
            plt.plot(df['close'].iloc[-30:], label='Histórico (30 dias)', marker='o')
            plt.plot(forecast_df['forecast'], label=f'Previsão ({days} dias)',
                    color='red', linestyle='--', marker='o')
            plt.title(f'Previsão para {SYMBOL}', fontsize=16)
            plt.xlabel('Data')
            plt.ylabel('Preço ($)')
            plt.legend()
            plt.grid(alpha=0.3)
            plt.show()

            return forecast_df

        except Exception as e:
            raise Exception(f"Erro na previsão: {str(e)}")

# ======================
# 📌 3. EXECUÇÃO PRINCIPAL
# ======================
def main():
    print(f"\n{'='*50}")
    print(f"=== PREVISÃO DE AÇÕES {SYMBOL} COM ANÁLISE DE SENTIMENTOS ===")
    print(f"{'='*50}\n")

    try:
        # 1. Baixar dados históricos
        print("📊 Baixando dados históricos...")
        end_date = datetime.now()
        start_date = end_date - timedelta(days=DAYS_TO_FETCH)
        df = yf.download(SYMBOL, start=start_date, end=end_date)[['Close', 'Volume']]
        df.columns = ['close', 'volume']
        print(f"✅ Dados de {df.index[0].date()} a {df.index[-1].date()}")
        print(f"📈 Último preço: ${df['close'].iloc[-1]:.2f}\n")

        # 2. Preparar modelo
        print("🧠 Preparando modelo...")
        predictor = StockPredictor()
        X_train, X_test, y_train, y_test, df_processed = predictor.prepare_data(df)
        print(f"📊 Dados preparados | Treino: {len(X_train)} | Teste: {len(X_test)}")

        # 3. Treinar modelo
        print("\n🔮 Treinando modelo...")
        cost_history = predictor.train(X_train, y_train)

        # Plot de convergência
        plt.figure(figsize=(12, 6))
        plt.plot(cost_history)
        plt.title('Convergência do Treinamento')
        plt.xlabel('Épocas')
        plt.ylabel('Erro (MSE)')
        plt.show()

        # 4. Avaliar modelo
        print("\n📊 Avaliando desempenho...")
        metrics = predictor.evaluate(X_test, y_test)
        print(f"\n🔍 Resultados:")
        print(f"- Erro Médio Absoluto (MAE): ${metrics['mae']:.2f}")
        print(f"- Raiz do Erro Quadrático Médio (RMSE): ${metrics['rmse']:.2f}")
        print(f"- Coeficiente de Determinação (R²): {metrics['r2']:.2%}")

        # 5. Fazer previsões
        print("\n🔮 Gerando previsões para os próximos 7 dias...")
        forecast = predictor.forecast(df_processed)
        print("\n📈 Previsões de Preço:")
        print(forecast.round(2))

    except Exception as e:
        print(f"\n❌ Erro: {str(e)}")
        print("\n💡 Possíveis soluções:")
        print("- Verifique sua conexão com a internet")
        print("- Confira se a API Key foi configurada corretamente")
        print("- Reduza o número de épocas se o treinamento estiver demorando")

if __name__ == "__main__":
    # Verifica e instala dependências se necessário
    try:
        import yfinance
    except ImportError:
        print("Instalando dependências necessárias...")
        !pip install yfinance nltk scikit-learn --quiet

    main()

✅ API Key carregada com sucesso dos Secrets do Colab

=== PREVISÃO DE AÇÕES NVDA COM ANÁLISE DE SENTIMENTOS ===

📊 Baixando dados históricos...


[*********************100%***********************]  1 of 1 completed


✅ Dados de 2024-04-03 a 2025-04-02
📈 Último preço: $110.42

🧠 Preparando modelo...

🔍 Buscando notícias reais sobre NVIDIA...
⚠️ Erro ao acessar API: 426 Client Error: Upgrade Required for url: https://newsapi.org/v2/everything?q=NVIDIA&language=en&pageSize=1000&apiKey=5f4f653a4023477fa7725ea18a221b5d

📰 Análise de Sentimento das Notícias:
📉 NEGATIVO (-0.30): NVIDIA anuncia avanços em IA com novos chips...
➖ NEUTRO (0.00): Ações da NVIDIA em alta após resultados positivos...
➖ NEUTRO (0.00): NVIDIA enfrenta desafios na cadeia de suprimentos...

🔍 Sentimento Médio: -0.10 (Escala: -1 a 1)
📊 Dados preparados | Treino: 184 | Teste: 46

🔮 Treinando modelo...
Época 0: MSE = 15856.0913
Época 1000: MSE = 17.8089
Época 2000: MSE = 17.4362
Época 3000: MSE = 17.3671
Época 4000: MSE = 17.3519
Época 5000: MSE = 17.3483
Época 6000: MSE = 17.3474
Época 7000: MSE = 17.3472
Época 8000: MSE = 17.3472
Época 9000: MSE = 17.3472
Época 10000: MSE = 17.3472
Época 11000: MSE = 17.3472
Época 12000: MSE = 17.34